In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel("TATASTEEL.xlsx", engine='openpyxl')
df

,Unnamed: 0,timestamp,open,high,low,close,volume
0,0,2021-08-23 14:11:00+05:30,1380.35,1381.00,1379.35,1380.25,15564
1,1,2021-08-23 14:12:00+05:30,1380.50,1381.00,1379.25,1379.80,28503
2,2,2021-08-23 14:13:00+05:30,1379.80,1380.55,1378.05,1378.80,29879
3,3,2021-08-23 14:14:00+05:30,1378.80,1378.80,1377.15,1378.00,16814
4,4,2021-08-23 14:15:00+05:30,1378.30,1378.70,1375.35,1376.25,36478
...,...,...,...,...,...,...,...
8620,8620,2021-09-24 14:06:00+05:30,1261.95,1262.70,1260.00,1260.15,65604
8621,8621,2021-09-24 14:07:00+05:30,1260.15,1261.00,1259.05,1259.70,107859
8622,8622,2021-09-24 14:08:00+05:30,1259.95,1260.35,1258.45,1259.95,54899
8623,8623,2021-09-24 14:09:00+05:30,1260.20,1261.85,1259.25,1261.65,49614


In [6]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace = True)
df

,open,high,low,close,volume
timestamp,,,,,
2021-08-23 14:11:00+05:30,1380.35,1381.00,1379.35,1380.25,15564
2021-08-23 14:12:00+05:30,1380.50,1381.00,1379.25,1379.80,28503
2021-08-23 14:13:00+05:30,1379.80,1380.55,1378.05,1378.80,29879
2021-08-23 14:14:00+05:30,1378.80,1378.80,1377.15,1378.00,16814
2021-08-23 14:15:00+05:30,1378.30,1378.70,1375.35,1376.25,36478
...,...,...,...,...,...
2021-09-24 14:06:00+05:30,1261.95,1262.70,1260.00,1260.15,65604
2021-09-24 14:07:00+05:30,1260.15,1261.00,1259.05,1259.70,107859
2021-09-24 14:08:00+05:30,1259.95,1260.35,1258.45,1259.95,54899


In [8]:
from datetime import datetime, timedelta
from kiteconnect import exceptions
import logging
import kiteconnect
from kiteconnect import KiteConnect
import pdb
import pandas as pd
import math
from numba import jit
import random

In [10]:
cash = input('Input the cash you want to assume during backtesting - ')
brokerage = input('Input the brokerage you want to assume during backtesting - ')

In [11]:
import backtrader as bt

class SmaCross(bt.Strategy):
    
    params = dict(
        pfast=7,  # period for the fast moving average
        pslow=35   # period for the slow moving average
        )
    
    def __init__(self):
        
        # ----------------------------------------------------------------------------------------
        # Declare all the variables required in the strategy here
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        
        
        # ----------------------------------------------------------------------------------------
        
        
        self.Statistics = pd.DataFrame(columns = ['Trade Open', 'Trade Close', 'Gross PnL', 'Net PnL', 'MaxDrawDown'])

    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        #print(f'{dt.isoformat()} {txt}') #Print date and close #Print date and close
    
#    @jit(nopython=True)
    def next(self):
        #self.log('Close, %.2f' % self.dataclose[0])
        
        # ----------------------------------------------------------------------------------------
        # Write the strategy logic here
        
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position
                
        # ----------------------------------------------------------------------------------------
        
    #@jit(nopython=True)
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
        # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

    # Check if an order has been completed
    # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None
        
    #@jit(nopython=True)
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        d1 = str(bt.num2date(trade.dtopen)) + '+05:30'
        d2 = str(bt.num2date(trade.dtclose)) + '+05:30'
        C = df['close']
        D = C.loc[d1:d2]
        maxdd = max_dd(D)
        if math.isnan(maxdd):
            self.Statistics.loc[len(self.Statistics.index)] = [d1, d2, trade.pnl, trade.pnlcomm, "Data Missing"]
        else:
            self.Statistics.loc[len(self.Statistics.index)] = [d1, d2, trade.pnl, trade.pnlcomm, maxdd]

#@jit(nopython=True)            
def max_dd(ser):
    max2here = ser.cummax()
    dd2here = (ser - max2here)/max2here
    return abs(dd2here.min()*100)

#@jit(nopython=True)
def printTradeAnalysis(analyzer):
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = round((total_won / total_closed) * 100, 2)
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

#@jit(nopython=True)        
def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

data = bt.feeds.PandasData(dataname=df[:2000])
cerebro.adddata(data)
#cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression = 10)
cerebro.broker.setcash(int(cash))
cerebro.broker.setcommission(commission = (float(brokerage))/100)

cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addsizer(bt.sizers.SizerFix, stake=3)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, compression = 1, factor=365, annualize=True)

if __name__ == '__main__':
    # Run Cerebro Engine
    start_portfolio_value = cerebro.broker.getvalue()

    strategies = cerebro.run()
    firstStrat = strategies[0]
    #b = Bokeh(style='bar', plot_mode='single')
    cerebro.plot()
    
    #print(firstStrat.Statistics)
    printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
    printSQN(firstStrat.analyzers.sqn.get_analysis())

    end_portfolio_value = cerebro.broker.getvalue()
    pnl = end_portfolio_value - start_portfolio_value
    print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
    print(f'Final Portfolio Value: {end_portfolio_value:2f}')
    print(f'PnL: {pnl:.2f}')
    print('Sharpe Ratio: ', firstStrat.analyzers.sharperatio.get_analysis()['sharperatio'])


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (C:\Users\LENOVO\anaconda3\lib\site-packages\matplotlib\dates.py)

In [12]:
firstStrat.Statistics['CumSum'] = firstStrat.Statistics['Net PnL'].cumsum()
firstStrat.Statistics

,Trade Open,Trade Close,Gross PnL,Net PnL,MaxDrawDown,CumSum
0,2021-08-24 03:45:00+05:30,2021-08-24 04:20:00+05:30,22.20,-1640.64,Data Missing,-1640.64
1,2021-08-24 04:33:00+05:30,2021-08-24 04:42:00+05:30,-10.95,-1683.06,Data Missing,-3323.70
2,2021-08-24 04:47:00+05:30,2021-08-24 05:01:00+05:30,-15.00,-1687.20,Data Missing,-5010.90
3,2021-08-24 05:09:00+05:30,2021-08-24 05:38:00+05:30,4.05,-1673.88,Data Missing,-6684.78
4,2021-08-24 05:44:00+05:30,2021-08-24 05:46:00+05:30,0.60,-1678.26,Data Missing,-8363.04
5,2021-08-24 06:33:00+05:30,2021-08-24 07:02:00+05:30,-5.85,-1676.22,Data Missing,-10039.26
6,2021-08-24 07:22:00+05:30,2021-08-24 08:36:00+05:30,47.70,-1635.66,Data Missing,-11674.92
7,2021-08-24 08:51:00+05:30,2021-08-24 09:23:00+05:30,-14.85,-1707.48,0.556057,-13382.40
8,2021-08-24 09:53:00+05:30,2021-08-24 09:58:00+05:30,0.75,-1685.04,0.03601,-15067.44
9,2021-08-25 03:46:00+05:30,2021-08-25 04:19:00+05:30,4.50,-1697.04,Data Missing,-16764.48


In [13]:
df2 = pd.DataFrame(np.array(firstStrat.Statistics['CumSum']), index = firstStrat.Statistics['Trade Close'], columns = ['CumSum'])
df2

,CumSum
Trade Close,
2021-08-24 04:20:00+05:30,-1640.64
2021-08-24 04:42:00+05:30,-3323.70
2021-08-24 05:01:00+05:30,-5010.90
2021-08-24 05:38:00+05:30,-6684.78
2021-08-24 05:46:00+05:30,-8363.04
2021-08-24 07:02:00+05:30,-10039.26
2021-08-24 08:36:00+05:30,-11674.92
2021-08-24 09:23:00+05:30,-13382.40
2021-08-24 09:58:00+05:30,-15067.44


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\LENOVO\anaconda3\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()


In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
df2.plot(linewidth=0.5)
plt.show()